## Step 1: Install Required Libraries
First, you need to install the necessary Python libraries:

In [2]:
!pip install yt-dlp pytube pandas webvtt-py requests


Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
import requests
import pandas as pd
import webvtt
from pytube import Playlist
import yt_dlp
import re
import time
import unicodedata

# Assume `playlist` is a pytube Playlist object already initialized
playlist_url = 'https://www.youtube.com/playlist?list=PL6XRrncXkMaU55GiCvv416NR2qBD_xbmf'
playlist = Playlist(playlist_url)

# Get all video URLs in the playlist
video_urls = [video.watch_url for video in playlist.videos]

# Directory to save thumbnails and transcripts
output_dir = 'data/en_CNN'
os.makedirs(output_dir, exist_ok=True)

def sanitize_filename(name):
    """
    Sanitize the filename by removing invalid characters, full-width symbols,
    stripping newlines, and normalizing unicode to ASCII where possible.
    """
    # Normalize full-width characters to their ASCII equivalents
    name = unicodedata.normalize('NFKC', name)  # Normalize Unicode characters to ASCII-compatible form
    
    # Remove problematic characters for filenames
    name = re.sub(r'[\\/*?:"<>|]', "", name)  # Remove invalid characters
    name = name.replace('\n', '').replace('\r', '')  # Remove newlines
    name = name.strip()  # Trim leading/trailing spaces

    # Ensure the filename length is not too long (you can set a specific limit)
    max_filename_length = 255
    return name[:max_filename_length]

def download_thumbnail(thumbnail_url, title):
    try:
        response = requests.get(thumbnail_url)
        sanitized_title = sanitize_filename(title)
        thumbnail_file = os.path.join(output_dir, f'{sanitized_title}.jpg')
        with open(thumbnail_file, 'wb') as file:
            file.write(response.content)
        return thumbnail_file
    except Exception as e:
        print(f"Failed to download thumbnail for {title}: {e}")
        return None

def download_subtitles(video_url, video_title, subtitle_languages):
    sanitized_title = sanitize_filename(video_title)
    
    # Define the download options
    ydl_opts = {
        'writesubtitles': True,
        'writeautomaticsub': True,
        'skip_download': True,  # Skip video download
        'outtmpl': os.path.join(output_dir, f'{sanitized_title}.%(ext)s'),  # Use sanitized title
        'restrictfilenames': True,
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            if 'en' in subtitle_languages:
                # Download English subtitles
                subtitle_file = os.path.join(output_dir, f'{sanitized_title}.en.vtt')
                if not os.path.exists(subtitle_file):
                    ydl_opts['subtitleslangs'] = ['en']
                    ydl.download([video_url])  # Download subtitles for the video
                return subtitle_file
            elif subtitle_languages:
                # Download the first available subtitle if English is not found
                first_lang = list(subtitle_languages)[0]
                subtitle_file = os.path.join(output_dir, f'{sanitized_title}.{first_lang}.vtt')
                if not os.path.exists(subtitle_file):
                    ydl_opts['subtitleslangs'] = [first_lang]
                    ydl.download([video_url])
                return subtitle_file
                
    except yt_dlp.utils.DownloadError as e:
        error_message = str(e)
        print(f"Error downloading subtitles for {video_title}: {error_message}")
        return None
                
def download_video_info(video_url, max_retries=3, retry_interval=180):
    ydl_opts = {
        'writesubtitles': True,
        'writeautomaticsub': True,
        'skip_download': True,
        #'outtmpl': os.path.join(output_dir, f'{"%(sanitized_title)s"}.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegMetadata',
        }],
    }
    retries = 0
    while retries < max_retries:
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(video_url, download=False)
                video_title = info_dict.get('title', None)
                video_id = info_dict.get('id', None)
                thumbnail_url = info_dict.get('thumbnail', None)
                publish_date = info_dict.get('upload_date', None)  # Format: 'YYYYMMDD'
                subtitle_languages = info_dict.get('automatic_captions', {}).keys()

                # Download thumbnail
                thumbnail_file = None
                if thumbnail_url:
                    thumbnail_file = download_thumbnail(thumbnail_url, video_title)

                # Download subtitles
                subtitle_file = None
                subtitle_file = download_subtitles(video_url, video_title, subtitle_languages)
                
            # Return the collected video information
            return video_id, video_title, thumbnail_file, subtitle_file, publish_date

        except yt_dlp.utils.DownloadError as e:
            error_message = str(e)
            if "Sign in to confirm you’re not a bot" in error_message:
                retries += 1
                print(f"Error: {error_message}. Retrying in {retry_interval} seconds... (Attempt {retries}/{max_retries})")
                time.sleep(retry_interval)  # Wait before retrying
            else:
                print(f"Failed to download video info for {video_url}: {e}")
                return None, None, None, None, None

    # After max retries, log the failure and return None
    print(f"Max retries reached. Skipping video: {video_url}")
    return None, None, None, None, None

def vtt_to_text(vtt_file_path):
    try:
        text = ''
        lines = []
        deduplicated_lines = []
        for caption in webvtt.read(vtt_file_path):
            # Split the block into lines and deduplicate them within the block
            lines.extend(caption.text.strip().split('\n'))
            #print(caption.text.strip().split('\n'))
        for line in lines:
            if line and (not deduplicated_lines or line != deduplicated_lines[-1]):  # Avoid consecutive duplicates
                deduplicated_lines.append(line)
     
        text = "\n".join(deduplicated_lines)
    
    except Exception as e:
        print(f"Failed to convert {vtt_file_path} to text: {e}")
        
    return text

# Create a list to store video information
data = []

# Loop through each video URL and download the required data
for video_url in video_urls:
    video_id, video_title, thumbnail_file, subtitle_file, publish_date = download_video_info(video_url)
    # If video info is successfully retrieved
    if video_id and video_title:
        # Convert subtitles to text if available
        transcript = None
        if subtitle_file and os.path.exists(subtitle_file):
            transcript = vtt_to_text(subtitle_file)
        else:
            print("subtitle file not found")
        # Append data to the list
        data.append({
            'video_id': video_id,
            'video_title': video_title,
            'thumbnail_path': thumbnail_file,
            'transcript': transcript,
            'publish_date': publish_date
        })

# Convert the list to a DataFrame
df = pd.DataFrame(data, columns=['video_id', 'video_title', 'thumbnail_path', 'transcript', 'publish_date'])

# Save the DataFrame to a CSV file
df.to_csv(os.path.join(output_dir,"../", 'en_CNN.csv'), index=False)

print(f"Dataset created successfully with {len(df)} entries.")

[youtube] Extracting URL: https://youtube.com/watch?v=hH8eMkO1CpA
[youtube] hH8eMkO1CpA: Downloading webpage
[youtube] hH8eMkO1CpA: Downloading ios player API JSON
[youtube] hH8eMkO1CpA: This video is age-restricted; some formats may be missing without authentication. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies
[youtube] hH8eMkO1CpA: Downloading tv embedded player API JSON
[youtube] hH8eMkO1CpA: Downloading web creator player API JSON
[info] hH8eMkO1CpA: Downloading subtitles: en


[youtube] Extracting URL: https://youtube.com/watch?v=L_6GMzVl6L0
[youtube] L_6GMzVl6L0: Downloading webpage
[youtube] L_6GMzVl6L0: Downloading ios player API JSON
[youtube] L_6GMzVl6L0: Downloading web creator player API JSON
[youtube] L_6GMzVl6L0: Downloading m3u8 information


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=kngd0K2KIqI
[youtube] kngd0K2KIqI: Downloading webpage
[youtube] kngd0K2KIqI: Downloading ios player API JSON
[youtube] kngd0K2KIqI: This video is age-restricted; some formats may be missing without authentication. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies
[youtube] kngd0K2KIqI: Downloading tv embedded player API JSON
[youtube] kngd0K2KIqI: Downloading web creator player API JSON


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=zdq3NVtT6Nc
[youtube] zdq3NVtT6Nc: Downloading webpage
[youtube] zdq3NVtT6Nc: Downloading ios player API JSON
[youtube] zdq3NVtT6Nc: Downloading web creator player API JSON
[youtube] zdq3NVtT6Nc: Downloading m3u8 information
[info] zdq3NVtT6Nc: Downloading subtitles: en


[youtube] Extracting URL: https://youtube.com/watch?v=p5ZTGkhmSJw
[youtube] p5ZTGkhmSJw: Downloading webpage
[youtube] p5ZTGkhmSJw: Downloading ios player API JSON
[youtube] p5ZTGkhmSJw: Downloading web creator player API JSON
[youtube] p5ZTGkhmSJw: Downloading m3u8 information


...

subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=BVL_czwbeeE
[youtube] BVL_czwbeeE: Downloading webpage
[youtube] BVL_czwbeeE: Downloading ios player API JSON
[youtube] BVL_czwbeeE: Downloading web creator player API JSON
[youtube] BVL_czwbeeE: Downloading m3u8 information


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=hrqXnnPe4FQ
[youtube] hrqXnnPe4FQ: Downloading webpage
[youtube] hrqXnnPe4FQ: Downloading ios player API JSON
[youtube] hrqXnnPe4FQ: Downloading web creator player API JSON
[youtube] hrqXnnPe4FQ: Downloading m3u8 information
[info] hrqXnnPe4FQ: Downloading subtitles: en


[youtube] Extracting URL: https://youtube.com/watch?v=hrqXnnPe4FQ
[youtube] hrqXnnPe4FQ: Downloading webpage
[youtube] hrqXnnPe4FQ: Downloading ios player API JSON
[youtube] hrqXnnPe4FQ: Downloading web creator player API JSON
[youtube] hrqXnnPe4FQ: Downloading m3u8 information
[info] hrqXnnPe4FQ: Downloading subtitles: en


[info] hrqXnnPe4FQ: Downloading 1 format(s): 18
[info] Writing video subtitles to: data\en_CNN\'They told us this area was safe' Designated Gaza safe zone targeted by Israeli airstrike.en.vtt
[download] Destination: data\en_CNN\'They told us this area was safe' Designated Gaza safe zone targeted by Israeli airstrike.en.vtt
[download] 100% of   12.82KiB in 00:00:00 at 87.63KiB/s
[youtube] Extracting URL: https://youtube.com/watch?v=yNmfu9CsTOk
[youtube] yNmfu9CsTOk: Downloading webpage
[youtube] yNmfu9CsTOk: Downloading ios player API JSON
[youtube] yNmfu9CsTOk: Downloading web creator player API JSON
[youtube] yNmfu9CsTOk: Downloading m3u8 information
[info] Testing format 616


subtitle file not found
Dataset created successfully with 370 entries.
